## Library Imports

In [1]:
import torch
import transformers
import onnx
import onnxruntime.training.onnxblock as onnxblock
from datasets import load_dataset
import json
import random
import re
from transformers import MobileBertConfig

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Generating artifacts

This section generates the necessary training artifacts: the training version of the ONNX model, the evaluation version of the ONNX model, and the optimizer.

In [10]:
config = MobileBertConfig(num_hidden_layers=2)
model = transformers.MobileBertForMaskedLM.from_pretrained('google/mobilebert-uncased', config=config)
# model = transformers.AutoModel.from_pretrained('google/mobilebert-uncased')
model_name = 'mobilebert-uncased'

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForMaskedLM: ['mobilebert.encoder.layer.3.attention.self.key.weight', 'mobilebert.encoder.layer.4.output.bottleneck.LayerNorm.bias', 'mobilebert.encoder.layer.9.ffn.0.output.LayerNorm.weight', 'mobilebert.encoder.layer.14.output.bottleneck.LayerNorm.bias', 'mobilebert.encoder.layer.7.bottleneck.input.dense.weight', 'mobilebert.encoder.layer.15.ffn.2.output.LayerNorm.weight', 'mobilebert.encoder.layer.20.output.bottleneck.LayerNorm.bias', 'mobilebert.encoder.layer.18.bottleneck.input.dense.bias', 'mobilebert.encoder.layer.9.intermediate.dense.weight', 'mobilebert.encoder.layer.16.bottleneck.attention.dense.weight', 'mobilebert.encoder.layer.17.output.LayerNorm.bias', 'mobilebert.encoder.layer.20.ffn.2.intermediate.dense.bias', 'mobilebert.encoder.layer.2.attention.output.LayerNorm.bias', 'mobilebert.encoder.layer.4.ffn.0.output.dense.bias', 'mobilebert.encoder.layer.9.ffn.0.interm

In [3]:
tokenizer = transformers.AutoTokenizer.from_pretrained("google/mobilebert-uncased")
inputs = tokenizer("The capital of France is [MASK].", return_tensors="pt")
labels = tokenizer("The capital of France is Paris.", return_tensors="pt")["input_ids"]
labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

In [11]:
class FlatModel(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, *local_inputs):
        return self.model(inputs.input_ids, inputs.attention_mask, inputs.token_type_ids, labels=labels)

model = FlatModel(model)

In [5]:
torch.onnx.export(
    model,
    (inputs["input_ids"], 
      inputs["attention_mask"],
      inputs["token_type_ids"],
      labels),
    f"training_artifacts/{model_name}.onnx",
    input_names=["input_ids", "attention_mask", "token_type_ids", "labels"],
    output_names=["loss", "logits"],
    dynamic_axes={
        "input_ids": {0: "batch_size", 1: "sequence_length"},
        "attention_mask": {0: "batch_size", 1: "sequence_length"},
        "token_type_ids": {0: "batch_size", 1: "sequence_length"},
        "labels": {0: "batch_size", 1: "sequence_length"},
        "logits ": {0: "batch_size", 1: "sequence_length"}
    },
    export_params=True,
    do_constant_folding=False,
    training=torch.onnx.TrainingMode.TRAINING,
)

/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:2029: UserWarning: Provided key logits  for dynamic axes is not a valid input/output name
  warnings.warn(
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/transformers/models/mobilebert/modeling_mobilebert.py:547: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  torch.tensor(1000),
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:967: UserWarning: Warning: ONNX export of embedding with padding_idx >= 0 for training mode. ONNX does not support not updating the embedding vector at padding_idx during training.
  warnings.warn(
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/_i

============= Diagnostic Run torch.onnx.export version 2.0.0+cu117 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:689: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(
/bert_ort/carolinezhu/newe2edemos/lib/python3.9/site-packages/torch/onnx/utils.py:1186: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at ../torch/csrc/jit/passes/onnx/constant_fold.cpp:179.)
  _C._jit_pass_onnx_graph_shape_type_inference(


In [12]:
from onnxruntime.training import artifacts
import onnx

requires_grad = []
frozen_params = []
for name, param in model.named_parameters():
    if param.requires_grad:
        requires_grad.append(name)
    else:
        frozen_params.append(name)

for name, param in model.named_buffers():
    frozen_params.append(name)

model = onnx.load(f"training_artifacts/{model_name}.onnx")


artifacts.generate_artifacts(
    model,
    requires_grad=requires_grad,
    frozen_params=frozen_params,
    optimizer=artifacts.OptimType.AdamW,
    artifact_directory="training_artifacts/"
)


2023-04-07 17:18:39.269827432 [I:onnxruntime:Default, graph.cc:3546 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/cls/predictions/transform/LayerNorm/Constant_output_0'. It is no longer used by any node.
2023-04-07 17:18:39.269878931 [I:onnxruntime:Default, graph.cc:3546 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/cls/predictions/transform/LayerNorm/Constant_1_output_0'. It is no longer used by any node.
2023-04-07 17:18:39.274630554 [I:onnxruntime:Default, graph.cc:3546 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/mobilebert/embeddings/Transpose_output_0'. It is no longer used by any node.
2023-04-07 17:18:39.274645154 [I:onnxruntime:Default, graph.cc:3546 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/mobilebert/embeddings/Slice_2_output_0'. It is no longer used by any node.
2023-04-07 17:18:39.274650454 [I:onnxruntime:Default, graph.cc:3546 CleanUnusedInitializersAndNodeArgs] Removing initializer '/model/m

## Generating tokens

This section tokenizes the dataset and then writes it into JSON files.

In [7]:
def tokenize_function(examples, pad_to_len):
    tokenizer = transformers.AutoTokenizer.from_pretrained("google/mobilebert-uncased")
    # filter out empty strings to remove unnecessary processing
    examples["text"] = [sent for sent in examples["text"] if len(sent) > 0]
    labels = tokenizer(examples["text"], padding="max_length", max_length=pad_to_len, truncation=True, return_tensors="pt")
    masked_examples = [mask(sent, pad_to_len) for sent in examples["text"]]
    inputs = tokenizer(masked_examples, padding="max_length", max_length=pad_to_len, truncation=True, return_tensors="pt")
    labels = torch.where(inputs["input_ids"] == tokenizer.mask_token_id, labels["input_ids"], -100)
    inputs["labels"] = labels
    return inputs

def mask(sent, pad_to_len):
    sent_words = sent.split()
    mask_index = random.randint(0, min(len(sent_words), pad_to_len) - 1)
    # replace random index with mask word, leaving punctuation as is
    # ... this preprocessing means that the token masked might be the <unk> word
    masked_words = [sent_words[ind] if ind != mask_index else re.sub("[a-zA-Z']+", "[MASK]", sent_words[ind]) for ind in range(len(sent_words))]
    return ' '.join(masked_words)

def generate_tokens(corpus):
    """
    Takes in a Dataset with a "text" feature.

    Returns a Dataset with the following features: text, input_ids, token_type_ids, attention_mask, special_tokens_mask
    """
    # pad_to_len must be calculated before the batching happens to create consistent sizes in the resulting tensor
    # pad_to_len = max([len(sent) for sent in corpus["text"]])
    pad_to_len = 80 # shortened for demonstration purposes
    return corpus.map(tokenize_function, batched=True, fn_kwargs={"pad_to_len": pad_to_len})

def generate_json_dict(token_dataset):
    """
    Takes in a Dataset with the following features: text, input_ids, token_type_ids, attention_mask, special_tokens_mask

    Basically changes the 2d Python lists into two fields: a shape & a flattened list, for easier conversion to OnnxValues

    Returns a dictionary with the following keys: input_ids, input_size, token_type_ids, token_type_size, attention_mask, attention_mask_size, special_tokens_mask, special_tokens_size
    """
    json_dict = {}
    keys_to_convert = ["input_ids", "token_type_ids", "attention_mask", "labels"]

    for key_name in keys_to_convert:
        # add field for the shape of the tensor
        json_dict[key_name + "_shape"] = [len(token_dataset[key_name]), len(token_dataset[key_name][0])]
        # flatten list
        json_dict[key_name] = [num for sent in token_dataset[key_name] for num in sent]
    
    return json_dict


In [8]:
dataset_name = "wikitext" 
dataset_config = "wikitext-2-v1"
# corpus = type DatasetDict with three Datasets: test, train, validation
corpus = load_dataset(dataset_name, dataset_config)

100%|██████████| 3/3 [00:00<00:00, 693.92it/s]


In [9]:
test_tokens_dataset = generate_tokens(corpus["test"])
test_tokens = generate_json_dict(test_tokens_dataset)
train_tokens_dataset = generate_tokens(corpus["train"])
train_tokens = generate_json_dict(train_tokens_dataset)
validation_tokens_dataset = generate_tokens(corpus["validation"])
validation_tokens = generate_json_dict(validation_tokens_dataset)

In [85]:
# write all the tokens to a json file
file_names = ["test_tokens.json", "train_tokens.json", "validation_tokens.json"]
token_dicts = [test_tokens, train_tokens, validation_tokens]

def write_dicts_to_files(file_names, dicts):
    # assumes file_names and dicts are 2 lists w/ the same lengths
    for i in range(len(file_names)):
        with open(file_names[i], "w") as json_file:
            json.dump(dicts[i], json_file)

write_dicts_to_files(file_names, token_dicts)